In [2]:
import pandas as pd 
import numpy as np 
transactions = pd.read_csv("data/transactions.csv")
transactions["day"] = pd.to_datetime(transactions["day"], dayfirst=False)

## Preparing data 
Filter on a single product for now, preparing a dataframe that can be used to fit a demand function

In [31]:
product = "Biologische brocolli"

filtered_transactions = transactions[(transactions["description"] == product)]
filtered_transactions["hour"] = pd.to_timedelta(filtered_transactions["time"])

def include_hour(row):
    hour = pd.to_datetime(row["time"]).hour
    return hour
filtered_transactions["hour"] = filtered_transactions.apply(include_hour, axis = 1)
filtered_transactions

<ipython-input-31-a999c74175f0>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-31-a999c74175f0>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic,hour
237,2018-01-01,13:03:28,16.0,NaN,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,13.0
420,2018-01-01,14:09:57,24.0,55280334.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,14.0
683,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,16.0
699,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,16.0
1057,2018-01-01,19:59:10,58.0,53245671.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
353636,2018-12-29,17:25:54,20625.0,90982137.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,17.0
353638,2018-12-29,17:25:54,20625.0,90982137.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,17.0
353642,2018-12-29,17:25:54,20625.0,90982137.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,17.0
353908,2018-12-30,9:09:23,20639.0,6301858.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0,9.0


In [32]:
purchases = filtered_transactions.groupby(["day", "hour"]).describe()["purchase_price"]
purchases

count  mean  std   min   25%   50%   75%   max
day        hour                                                
2018-01-01 13.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
           14.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
           16.0    2.0  1.79  0.0  1.79  1.79  1.79  1.79  1.79
           19.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
2018-01-02 12.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
...                ...   ...  ...   ...   ...   ...   ...   ...
2018-12-29 13.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
           15.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
           17.0    3.0  1.79  0.0  1.79  1.79  1.79  1.79  1.79
2018-12-30 9.0     1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79
           16.0    1.0  1.79  NaN  1.79  1.79  1.79  1.79  1.79

[932 rows x 8 columns]

In [18]:
days = [purchases.index[i][0].dayofyear for i in range(purchases.shape[0])]
hours = [purchases.index[i][1] for i in range(purchases.shape[0])]
dayofweek = [purchases.index[i][0].dayofweek + 1 for i in range(purchases.shape[0])]
purchases["hour"] = hours
purchases["day"] = days
purchases["dayofweek"] = dayofweek
pred_cols = ["min", "max", "hour", "day", "dayofweek"]
target_col = "count"

purchases

count  mean  std   min   25%   50%   75%   max  hour  day  \
day        hour                                                              
2018-01-01 10.0    4.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  10.0    1   
           11.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  11.0    1   
           13.0    6.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  13.0    1   
           14.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  14.0    1   
           15.0    5.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  15.0    1   
...                ...   ...  ...   ...   ...   ...   ...   ...   ...  ...   
2018-12-30 15.0    5.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  15.0  364   
           16.0    1.0  1.29  NaN  1.29  1.29  1.29  1.29  1.29  16.0  364   
           17.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  17.0  364   
           19.0    2.0  1.29  0.0  1.29  1.29  1.29  1.29  1.29  19.0  364   
           20.0    1.0  1.29  NaN  1.29  1.29  1.29  1.29  1.29  20.0  364   

                 dayofweek  
day        hour             
2018-01-01 10.0          1  
           11.0          1  
           13.0          1  
           14.0          1  
           15.0          1  
...                    ...  
2018-12-30 15.0          7  
           16.0          7  
           17.0          7  
           19.0          7  
           20.0          7  

[3659 rows x 11 columns]

## Alternative to group by daily purchases

In [33]:
purchases = filtered_transactions.groupby("day").describe()["purchase_price"]
purchases["dayofweek"] = [purchases.index[i].dayofweek +1 for i in range(purchases.shape[0])]
purchases["day"] = [purchases.index[i].dayofyear for i in range(purchases.shape[0])]
purchases

,count,mean,std,min,25%,50%,75%,max,dayofweek,day
day,,,,,,,,,,
2018-01-01,5.0,1.79,2.482534e-16,1.79,1.79,1.79,1.79,1.79,1,1
2018-01-02,1.0,1.79,NaN,1.79,1.79,1.79,1.79,1.79,2,2
2018-01-03,1.0,1.79,NaN,1.79,1.79,1.79,1.79,1.79,3,3
2018-01-04,5.0,1.79,2.482534e-16,1.79,1.79,1.79,1.79,1.79,4,4
2018-01-05,4.0,1.79,0.000000e+00,1.79,1.79,1.79,1.79,1.79,5,5
...,...,...,...,...,...,...,...,...,...,...
2018-12-26,5.0,1.79,2.482534e-16,1.79,1.79,1.79,1.79,1.79,3,360
2018-12-27,1.0,1.79,NaN,1.79,1.79,1.79,1.79,1.79,4,361
2018-12-28,1.0,1.79,NaN,1.79,1.79,1.79,1.79,1.79,5,362


In [36]:
import plotly.express as px
#plotting per day
fig = px.scatter(purchases, y="count", x="day", color="mean")
fig.show()

## Implementing a ML model for demand function

In [65]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(purchases)
train.dropna(inplace=True)
test.dropna(inplace=True)
X_train = train[pred_cols].values
y_train = train[target_col].values

X_test = test[pred_cols].values
y_test = test[target_col].values

<ipython-input-65-8481154c4a32>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-65-8481154c4a32>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



True

In [66]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg.score(X_test, y_test)

0.03224311938423663

In [13]:
purchases = filtered_transactions.groupby("day").describe()["purchase_price"]



In [19]:
fig = px.scatter(purchases, x="hour", y="count", color="mean")
fig.show()

In [11]:
purchases[purchases["std"] > 0.001]

count      mean       std   min     25%    50%     75%   max  \
day        hour                                                                 
2018-02-26 9.0    10.0  0.975000  0.217371  0.84  0.8400  0.840  1.1775  1.29   
           10.0    5.0  1.110000  0.246475  0.84  0.8400  1.290  1.2900  1.29   
2018-03-12 9.0     5.0  1.254000  0.032863  1.23  1.2300  1.230  1.2900  1.29   
2018-05-03 9.0     4.0  1.177500  0.225000  0.84  1.1775  1.290  1.2900  1.29   
2018-06-08 9.0     6.0  1.023333  0.130639  0.97  0.9700  0.970  0.9700  1.29   
           10.0    9.0  1.005556  0.106667  0.97  0.9700  0.970  0.9700  1.29   
           19.0    5.0  1.034000  0.143108  0.97  0.9700  0.970  0.9700  1.29   
2018-08-13 9.0     3.0  1.183333  0.184752  0.97  1.1300  1.290  1.2900  1.29   
           10.0    7.0  1.198571  0.156144  0.97  1.1300  1.290  1.2900  1.29   
2018-10-15 9.0     3.0  1.160000  0.225167  0.90  1.0950  1.290  1.2900  1.29   
           11.0    3.0  1.160000  0.225167  0.90  1.0950  1.290  1.2900  1.29   
2018-10-22 10.0    3.0  1.160000  0.225167  0.90  1.0950  1.290  1.2900  1.29   
           11.0    2.0  1.095000  0.275772  0.90  0.9975  1.095  1.1925  1.29   
2018-12-17 11.0    5.0  1.266000  0.032863  1.23  1.2300  1.290  1.2900  1.29   

                 hour  day  dayofweek  
day        hour                        
2018-02-26 9.0    9.0   57          1  
           10.0  10.0   57          1  
2018-03-12 9.0    9.0   71          1  
2018-05-03 9.0    9.0  123          4  
2018-06-08 9.0    9.0  159          5  
           10.0  10.0  159          5  
           19.0  19.0  159          5  
2018-08-13 9.0    9.0  225          1  
           10.0  10.0  225          1  
2018-10-15 9.0    9.0  288          1  
           11.0  11.0  288          1  
2018-10-22 10.0  10.0  295          1  
           11.0  11.0  295          1  
2018-12-17 11.0  11.0  351          1